**Numerical Methods Coursework 1**

In [1]:
import numpy as np

Firstly we implement the Kamrad-Ritchken model.

In [7]:

class TrinomialTreeKamradRitchken():
    def __init__(r, dt, sigma, s0, N, lamda):
        # CRR parameters
        self._dt = dt
        self._r = r
        self._u=np.exp(lamda*sigma*np.sqrt(dt))
        self._qu = 1/(2*lamda**2) + ((r - sigma**2/2)*np.sqrt(dt))/2*lamda*sigma
        self._qm = 1-1/lamda**2
        self._qd = 1/(2*lamda**2) - ((r - sigma**2/2)*np.sqrt(dt))/2*lamda*sigma

        if (self._qu < 0 or self._qd < 0):
            print("The value of dt is too large and the probabilities will not be viable, please use a smaller value for dt.")

        # Set up the tree via S[k,n] = s0 * u^(2k-n)
        self._prices = np.zeros((N+1, N+1))
        for t in range(N+1):
            self._prices[0:2*t, t] = s0 * u**np.arange(-t, t, 1)

    def get_stock_price(n,k):
        return self._prices[k,n]

    def discount(x, number_of_periods):
        return x*np.exp(-1*self._r*self._dt*number_of_periods)

everything from here below is unchanged from the exercise sheet.

*Step 2: pick an auxiliary variable*

We will choose $A_n=\frac{1}{n+1}S_n$, the running average up to time $n$, to be our auxiliary process. In the programming, we can keep the payoff function to be generic as something in form of $g(S_N,A_N)$, and then we pass the whole payoff function to the pricing algorithm.

*Step 3: Specify a grid of values for the auxiliary variable $A_n$*

We follow the procedures in the Lecture Notes (page 20-21) to construct a matrix which contains the values $a_j^n$.

In [2]:
def CreateGridA(u, d, rho, N):

    # the array of J_min^n, J_max^n
    JMax = np.zeros(N+1)
    JMin = np.zeros(N+1)

    # Set each value of J_min^n and J_max^n inductively
    for n in range(1,N+1):
        JMax[n] = int(np.ceil(np.log((n*u**(rho*JMax[n-1]) + u**n) / (n+1)) / (rho*np.log(u))))
        JMin[n] = int(np.ceil(-1*np.log((n*u**(-1*rho*JMin[n-1]) + d**n) / (n+1)) / (rho*np.log(u))))

    # Force the types of variables inside the Jmin Jmax vectors be integers
    # (it should have already been done in the previous lines but my code is still complaining about the variable type...)
    JMax = JMax.astype(int)
    JMin = JMin.astype(int)

    avgSize = JMax[N] + JMin[N] + 1 # maximum number of grid points along the average value dimension
    
    # fill in the values of the grid a_j^n
    avg = np.zeros((avgSize, N+1))
    for n in range(N+1):
        j_range = np.arange(-1 * JMin[n] , JMax[n] + 1) #can't use negative index in programming so need to shift the index j via J_max^N - j to ensure the index starts from 0
        avg[JMax[N] - j_range, n] = s0 * u**(rho * j_range) 

    return avg, JMax, JMin, avgSize

*Step 4: implementation of the backward induction*

In [3]:
def AsianOptionPricing(s0, r, dt, sigma, N, rho, payoff):
               
    S, u, d, qu, qd, R = BinomialTreeCRR(r, dt, sigma, s0, N)
    avg, JMax, JMin, avgSize = CreateGridA(u, d, rho, N)
            
    # Option values matrix
    OptPx = np.zeros((N+1, avgSize, N+1)) # stock px, avg, time 
    
    # Terminal condition: option values given by terminal payoff
    for k in range(N+1):
        j_range = np.arange(-1 * JMin[N] , JMax[N] + 1)
        OptPx[k,int(JMax[N]) - j_range,N] = payoff(S[k,N], avg[JMax[N] - j_range,N])
            
    # Backward induction

    # Loop in time
    for n in reversed(range(N)):
 
        # Loop across each stock price index k
        for k in range(n+1):
            
            j_range = np.arange(-1 * JMin[n] , JMax[n] + 1)
            
            proj_avg_up = ((n+1)*avg[JMax[N] - j_range,n] + S[k,n+1])/(n+2)   # projected new avg value when stock goes up
            phi_up = np.log(proj_avg_up/s0) / (rho * np.log(u))               # the new index j obtained via shooting function (which is not integer)
            phi_up_ceil = (np.ceil(phi_up)).astype(int)                       # ceiling of the integer
            phi_up_floor = (np.floor(phi_up)).astype(int)                     # floor of the integer
            alpha_up = (np.log(avg[JMax[N]-phi_up_ceil,n+1]) - np.log(proj_avg_up)) / (rho*np.log(u))   # weighting to be used in the linear interpolation
            V_up = alpha_up * OptPx[k,JMax[N] - phi_up_floor,n+1] + (1 - alpha_up) *OptPx[k, JMax[N] - phi_up_ceil,n+1]  # apply interpolation to estimate the value of option in the upward case
            
            proj_avg_down = ((n+1)*avg[JMax[N] - j_range,n] + S[k+1,n+1])/(n+2)   # projected new avg value when stock goes down
            phi_down = np.log(proj_avg_down/s0) / (rho * np.log(u))               # the new index j obtained via shooting function (which is not integer)
            phi_down_ceil = (np.ceil(phi_down)).astype(int)                        # ceiling of the integer
            phi_down_floor = (np.floor(phi_down)).astype(int)                      # floor of the integer
            alpha_down = (np.log(avg[JMax[N]-phi_down_ceil,n+1]) - np.log(proj_avg_down)) / (rho*np.log(u))   # weighting to be used in the linear interpolation
            V_down = alpha_down * OptPx[k+1,JMax[N] - phi_down_floor,n+1] + (1 - alpha_down) * OptPx[k+1, int(JMax[N]) - phi_down_ceil,n+1]   # apply interpolation to estimate the value of option in the downward case
            
            # the backward induction step
            OptPx[k,int(JMax[N])-j_range,n] = (qu*V_up + (1-qu)*V_down) / R
    
    return S, avg, OptPx, OptPx[0,JMax[N],0] #the last component is the option value at time 0

*Finally let's implement the code and look at the numerical example in the Lecture Notes.*

In [12]:
s0, r, T, sigma, strike = 100, 0.01, 1, 0.2, 100
lamda = 1.5
N, rho = 1, 0.5
FixedStrikeAsianCall = lambda s,a: np.maximum(a-strike, 0) # payoff definition of a fixed strike asian call

dt = T/N

S, avg, OptPx, p = AsianOptionPricing(s0, r, dt, sigma, N, rho, FixedStrikeAsianCall)
print("Fair price of the option is", p)

Fair price of the option is 5.220130966346353
